In [482]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
# from seleniumwire import webdriver as wbw
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  
# from selenium.webdriver.common.by import By
# from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random
from urllib.request import urlopen
import urllib.parse
import numpy as np
from IPython.display import display
import datetime

In [513]:
def get_soup_driver(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(10 + random.uniform(-1,1))
    r = driver.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  

In [225]:
def import_df(path): # takes in file path of df
    try: 
        df = pd.read_csv(path, index_col=0)
        if 'overdrive_author' not in list(df.columns):
            df['overdrive_author'] = np.nan
        start_index = df[df.overdrive_author.isna()].iloc[0].name
    except FileNotFoundError:
        return 0, 0
    
    return int(start_index), df

In [227]:
def import_df_author(path): # takes in file path of df
    try: 
        df = pd.read_csv(path, index_col=0)
        if 'author_stars' not in list(df.columns):
            df['author_stars'] = np.nan
            df['author_ratings'] = np.nan
            df['author_num_reviews'] = np.nan
            df['author_distinct_works'] = np.nan
        start_index = df[df.author_stars.isna()].iloc[0].name
    except FileNotFoundError:
        return 0, 0
    
    return int(start_index), df

In [525]:
def import_df_author_redo(path): # takes in file path of df
    try:      
        df = pd.read_csv(path, index_col=0)
        if 'overdrive_author' not in list(df.columns):
            df['overdrive_author'] = np.nan        
        sub_df = df[(df.author_url.isna()) | ((df.author_stars.isna()) & (df.year_published.notna()))]
        indices = list(sub_df.index)
#         raise FileNotFoundError
    except FileNotFoundError:
        return [0], df
    
    return indices, df


# '''def import_df_redo(path): # takes in file path of df
#     try: 
#         df = pd.read_csv(path, index_col=0)
#         if 'overdrive_author' not in list(df.columns):
#             df['overdrive_author'] = np.nan
#         sub_df = df[(df['Library copies'] == 'info unavailable') | (df.overdrive_author.isna())]
#         indices = list(sub_df.index)
# #         print(start_index)
#     except FileNotFoundError:
#         return 0, 0
    
#     return indices, df'''

In [207]:
def get_pages_year_authorURL_GR(path):
    
    list_of_indices, df = import_df(path)
    base_url = 'https://www.goodreads.com/'

    position_start_index = list(df.index).index(start_index)
    list_of_indices = list(df.index[position_start_index:])
    
    try:
        df.insert(3, 'author_url', np.nan)
    except:
        pass
    
    for index in list_of_indices:
#         print('ding')
        soup = get_soup_proxy(base_url + df.loc[index, 'url'])
        
        try: num_pages = soup.find(itemprop='numberOfPages').text
        except: num_pages = -1
        
        try: year_published = soup.find(text = re.compile('Published')).text.replace('Published','').replace('\n', '').strip()
        except: year_published = -1
            
        try: author_url = soup.find('a', class_='authorName').get('href')
        except: author_url = -1
        
        df.loc[index, 'author_url'] = author_url      
        df.loc[index, 'number of pages'] = num_pages
        df.loc[index, 'year_published'] = year_published   

        
        df.to_csv(path)
        time.sleep(11 + random.uniform(-1,1))
        
#     return pd.DataFrame(list_results)
    

Manually change df for this to work

In [195]:
path = 'Figures/good_reads_df.csv'
df = pd.read_csv(path, index_col=0)

In [197]:
df = df.drop_duplicates(subset=['url'])
df

,id_number,title,author,stars,number of ratings,url,is_series,year_published,price,number of pages,...,movie title,movie year,overdrive_url,media,Overdrive title,Wait time,Library copies,People waiting in total,People waiting per copy,Genres
0,0,Night,Elie Wiesel,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,12.99,'NoneType' object has no attribute 'parent',...,Last Night in Soho,2021,/media/3946821,Audiobook,Night,0,5,-1,0,"['Biography & Autobiography', 'History', 'Nonf..."
2,1,The Westing Game,Ellen Raskin,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,8.99,192,...,The Westing Game,1997,/media/283610,ebook,The Westing Game,About 3 weeks,8,8,1,"['Juvenile Fiction', 'Juvenile Literature', 'M..."
4,2,Hunted,P.C. Cast,3.91 avg rating,"178,631 ratings",/book/show/4134071-hunted,True,"March 14, 2017",1.99,432,...,Good Will Hunting,1997,/media/5408082,ebook,"The Amazing Spider-Man by Nick Spencer, Volume 4",0,2,-2,0,"['Comic and Graphic Books', 'Fiction']"
6,3,We Need to Talk About Kevin,Lionel Shriver,4.04 avg rating,"188,422 ratings",/book/show/80660.We_Need_to_Talk_About_Kevin,False,"April 14, 2003",14.99,416,...,We Need to Talk About Kevin,2011,/media/602781,ebook,We Need to Talk About Kevin,0,12,-6,0,"['Fiction', 'Literature']"
8,4,The Bean Trees,Barbara Kingsolver,3.98 avg rating,"140,989 ratings",/book/show/30868.The_Bean_Trees,True,1988,7.99,232,...,NaN,NaN,/media/132411,ebook,The Bean Trees,0,3,-2,0,"['Fiction', 'Literature']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,995,The Notebook,Nicholas Sparks,4.12 avg rating,"1,464,216 ratings",/book/show/33648131-the-notebook,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1562,996,Winnie-the-Pooh,A.A. Milne,4.32 avg rating,"301,205 ratings",/book/show/99107.Winnie_the_Pooh,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1563,997,The Complete Stories and Poems,Edgar Allan Poe,4.37 avg rating,"233,410 ratings",/book/show/23919.The_Complete_Stories_and_Poems,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1564,998,Interview with the Vampire,Anne Rice,4.00 avg rating,"526,137 ratings",/book/show/43763.Interview_with_the_Vampire,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
df = df[(df.year_published == "'NoneType' object has no attribute 'parent'") | (df['number of pages'] == "'NoneType' object has no attribute 'parent'")]

In [202]:
df.to_csv('Figures/get_year_page_count.csv')

In [209]:
def scrape(path):
    error_counter = 0
    try: get_pages_year_authorURL_GR(path)
    except:
        print('error')
        time.sleep(60 + random.uniform(-2,2))
        error_counter += 1
        if error_counter > 3:
            return 'Terminated, check for errors'
        scrape(path)

In [210]:
scrape('Figures/get_year_page_count.csv')

In [505]:
def get_soup_proxy(url): # Retrieves soup from website using py requests module and 'person' headers
    API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
    base_url = f'https://api.webscrapingapi.com/v1?api_key={API_KEY}&url='
    url = base_url + urllib.parse.quote(url, safe='~')
    response = requests.get(url, timeout=(10,20))
    status = response.status_code
    if status == 200:
        page = response.text
        soup = bs(page)
    else:
        print(f"Oops! Received status code {status}")
    return soup

# def get_soup_proxy(url): # Retrieves soup from website using py requests module and 'person' headers
# #     API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
# #     base_url = f'https://api.webscrapingapi.com/v1?api_key={API_KEY}&url='
# #     url = base_url + urllib.parse.quote(url, safe='~')
# #     response = requests.get(url, timeout=(10,20))
# #     status = response.status_code
    
#     url = "https://api.webscrapingapi.com/v1"

#     params = {
#       "api_key":"OavaC96DqSNsO1vM2qStOhzTZoeOakNd",
#       "url":url
#     }
#     response = requests.request("GET", url, params=params, timeout=(10,20))
#     status = response.status_code

#     if status == 200:
#         page = response.text
#         soup = bs(page)
#     else:
#         print(f"Oops! Received status code {status}")
#     return soup

In [506]:
soup = get_soup_proxy('https://www.goodreads.com/book/show/6853.One_for_the_Money')

In [507]:
soup.find(itemprop='numberOfPages').text

'320 pages'

In [79]:
soup.find(text = re.compile('Published')).text.replace('Published','').replace('\n', '').strip()

'2006         by Griffin'

In [96]:
soup.find('a', class_='authorName').get('href')

'https://www.goodreads.com/author/show/2384.Janet_Evanovich'

In [90]:
new_df.drop_duplicates(subset=['url'], inplace=True)

/opt/anaconda3/envs/metis/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [504]:
soup = get_soup_proxy('https://www.nypl.overdrive.com')

ReadTimeout: HTTPSConnectionPool(host='api.webscrapingapi.com', port=443): Read timed out. (read timeout=20)

In [ ]:
soup

In [216]:
soup.find(class_='rating').text.replace('Average rating:', '').replace('\n','').strip()#replace

'4.02'

In [219]:
soup.find(class_='votes').text.replace('\n','').strip()#replace

'175,675'

In [220]:
soup.find(class_='count').text.replace('\n','').strip()#replace

'13,018'

In [224]:
soup.find(class_='hreview-aggregate').find('a').text

'53 distinct works'

In [261]:
def get_author_attr_GR(path):
    
    start_index, df = import_df_author(path)
    base_url = ''

    position_start_index = list(df.index).index(start_index)
    list_of_indices = list(df.index[position_start_index:])
       
    for index in list_of_indices:
#         print('ding')
#         print(base_url + df.loc[index, 'author_url'])
        if (base_url + df.loc[index, 'author_url'])[0] == 'h' and (df.loc[index, 'year_published'] == -1 \
            or df.loc[index, 'year_published'] == "'NoneType' object has no attribute 'parent'"):
       
            soup = get_soup_proxy(base_url + df.loc[index, 'author_url'])

            try: rating = soup.find(class_='rating').text.replace('Average rating:', '').replace('\n','').strip()#replace
            except: rating = -1

            try: num_ratings = soup.find(class_='votes').text.replace('\n','').strip()#replace
            except: num_ratings = -1

            try: num_reviews = soup.find(class_='count').text.replace('\n','').strip()#replace
            except: num_reviews = -1

            try: num_distinct_works = soup.find(class_='hreview-aggregate').find('a').text
            except: num_distinct_works = -1           


            df.loc[index, 'author_stars'] = rating      
            df.loc[index, 'author_ratings'] = num_ratings
            df.loc[index, 'author_num_reviews'] = num_reviews   
            df.loc[index, 'author_distinct_works'] = num_distinct_works


            df.to_csv(path)
            time.sleep(11 + random.uniform(-1,1))
        
#     return pd.DataFrame(list_results)
    

In [287]:
def get_author_attr_GR_redo(path): # takes path, imports df, gets list of rows that need operating on
    
    list_of_indices, df = import_df_author_redo(path)
    base_url = ''

#     print(list_of_indices) 
    
    for index in list_of_indices:
#         display(df[df.loc[:, 'year_published'] == '-1'])
#         break
#         print('ding')
#         print(base_url + df.loc[index, 'author_url'])
        if (base_url + df.loc[index, 'author_url'])[0] == 'h' and (df.loc[index, 'year_published'] == '-1' \
            or df.loc[index, 'year_published'] == "'NoneType' object has no attribute 'parent'"):
       
            soup = get_soup_proxy(base_url + df.loc[index, 'author_url'])

            try: rating = soup.find(class_='rating').text.replace('Average rating:', '').replace('\n','').strip()#replace
            except: rating = -1

            try: num_ratings = soup.find(class_='votes').text.replace('\n','').strip()#replace
            except: num_ratings = -1

            try: num_reviews = soup.find(class_='count').text.replace('\n','').strip()#replace
            except: num_reviews = -1

            try: num_distinct_works = soup.find(class_='hreview-aggregate').find('a').text
            except: num_distinct_works = -1           


            df.loc[index, 'author_stars'] = rating      
            df.loc[index, 'author_ratings'] = num_ratings
            df.loc[index, 'author_num_reviews'] = num_reviews   
            df.loc[index, 'author_distinct_works'] = num_distinct_works


            df.to_csv(path)
            time.sleep(11 + random.uniform(-1,1))
        
#     return pd.DataFrame(list_results)
    

In [249]:
def scrape_author(path, error_counter=0):
    error_counter = error_counter
    try: get_author_attr_GR(path)
    except:
        print('error')
        time.sleep(60 + random.uniform(-2,2))
        error_counter += 1
        if error_counter > 2:
            return 'Terminated, check for errors'
        scrape_author(path, error_counter)

In [265]:
def scrape_author_redo(path, error_counter=0):
    error_counter = error_counter
    try: get_author_attr_GR_redo(path)
    except:
        print('error')
        time.sleep(60 + random.uniform(-2,2))
        error_counter += 1
        if error_counter > 2:
            return 'Terminated, check for errors'
        scrape_author(path, error_counter)

In [519]:
def get_pages_year_authorURL_GR_attr_redo(path):
    
    list_of_indices, df = import_df_author_redo(path)
    base_url = 'https://www.goodreads.com'

#     position_start_index = list(df.index).index(start_index)
#     list_of_indices = list(df.index[position_start_index:])
    
#     try:
#         df.insert(3, 'author_url', np.nan)
#     except:
#         pass
    
    for index in list_of_indices:
#         print('ding')
        print(base_url + df.loc[index, 'url'])
#         soup = get_soup_proxy(base_url + df.loc[index, 'url'])
        soup = get_soup_driver(base_url + df.loc[index, 'url'])        
        
        try: num_pages = soup.find(itemprop='numberOfPages').text
        except: num_pages = -1
        
        try: year_published = soup.find(text = re.compile('Published')).text.replace('Published','').replace('\n', '').strip()
        except: year_published = -1
            
        try: author_url = soup.find('a', class_='authorName').get('href')
        except: author_url = -1
            
        print(num_pages, year_published)
        
        time.sleep(20 + random.uniform(-1,1))
        
#         if author_url != -1: soup = get_soup_proxy(author_url)
        if author_url != -1: 
            soup = get_soup_driver(author_url)

            try: rating = soup.find(class_='rating').text.replace('Average rating:', '').replace('\n','').strip()#replace
            except: rating = -1

            try: num_ratings = soup.find(class_='votes').text.replace('\n','').strip()#replace
            except: num_ratings = -1

            try: num_reviews = soup.find(class_='count').text.replace('\n','').strip()#replace
            except: num_reviews = -1

            try: num_distinct_works = soup.find(class_='hreview-aggregate').find('a').text
            except: num_distinct_works = -1           

        print(rating, num_ratings, num_reviews, num_distinct_works)
        
        df.loc[index, 'author_stars'] = rating      
        df.loc[index, 'author_ratings'] = num_ratings
        df.loc[index, 'author_num_reviews'] = num_reviews   
        df.loc[index, 'author_distinct_works'] = num_distinct_works
            
        
        df.loc[index, 'author_url'] = author_url      
        if df.loc[index, 'number of pages'] == '-1' or df.loc[index, 'number of pages'] == "'NoneType' object has no attribute 'parent'":
            df.loc[index, 'number of pages'] = num_pages
        
        if df.loc[index, 'year_published'] == '-1' or df.loc[index, 'year_published'] == "'NoneType' object has no attribute 'parent'":
            df.loc[index, 'year_published'] = year_published

        
        df.to_csv(path)
        time.sleep(20 + random.uniform(-1,1))
        
#     return pd.DataFrame(list_results)
    

In [515]:
def new_scrape_redo(path, error_counter=0):
    global driver
    driver = webdriver.Chrome()
    error_counter = error_counter
    try: 
        print(error_counter)
        get_pages_year_authorURL_GR_attr_redo(path)
    except:
        driver.quit()
        print('error')
        print(datetime.datetime.now())
        time.sleep(300 + random.uniform(-2,2))
        error_counter += 1
        if error_counter > 120:
            return 'Terminated, check for errors'
        new_scrape_redo(path, error_counter)

In [526]:
new_scrape_redo('Figures/merged_df.csv')

0
https://www.goodreads.com/book/show/46165.This_Side_of_Paradise
275 pages July 14th 1998         by Scribner
3.91 4,719,448 105,115 813 distinct works
https://www.goodreads.com/book/show/46165.This_Side_of_Paradise
275 pages July 14th 1998         by Scribner
3.91 4,719,448 105,115 813 distinct works
https://www.goodreads.com/book/show/11367726-defending-jacob
421 pages January 31st 2012         by Delacorte Press
3.97 293,413 22,835 7 distinct works
https://www.goodreads.com/book/show/11367726-defending-jacob
421 pages January 31st 2012         by Delacorte Press
3.97 293,413 22,835 7 distinct works
https://www.goodreads.com/book/show/7091488-dead-in-the-family
320 pages May 4th 2010         by Ace Books
3.93 3,536,522 135,768 206 distinct works
https://www.goodreads.com/book/show/7091488-dead-in-the-family
320 pages May 4th 2010         by Ace Books
3.93 3,536,522 135,768 206 distinct works
https://www.goodreads.com/book/show/6174.Survival_in_Auschwitz
187 pages September 1st 1995 

4.01 1,349,099 87,133 34 distinct works
https://www.goodreads.com/book/show/394469.Suttree
471 pages May 1992         by Vintage International
4.01 1,349,099 87,133 34 distinct works
https://www.goodreads.com/book/show/53138095-a-court-of-silver-flames
757 pages February 16th 2021         by Bloomsbury Publishing
4.34 5,785,778 492,393 82 distinct works
https://www.goodreads.com/book/show/53138095-a-court-of-silver-flames
757 pages February 16th 2021         by Bloomsbury Publishing
4.34 5,785,778 492,393 82 distinct works
https://www.goodreads.com/book/show/47624.Lirael
464 pages December 30th 2015         by Eos
4.03 872,391 45,061 201 distinct works
https://www.goodreads.com/book/show/47624.Lirael
464 pages December 30th 2015         by Eos
4.03 872,391 45,061 201 distinct works
https://www.goodreads.com/book/show/11966216-torn
324 pages February 28th 2012         by St. Martin's Griffin
3.93 454,636 31,494 58 distinct works
https://www.goodreads.com/book/show/11966216-torn
324 page

398 pages March 29th 2011         by St. Martin's Griffin
4.24 315,600 15,246 25 distinct works
https://www.goodreads.com/book/show/34053.Just_So_Stories
210 pages August 5th 2003         by Gramercy Books
3.92 399,704 17,207 1,599 distinct works
https://www.goodreads.com/book/show/34053.Just_So_Stories
210 pages August 5th 2003         by Gramercy Books
3.92 399,704 17,207 1,599 distinct works
https://www.goodreads.com/book/show/3393124-max
309 pages March 16th 2009         by jimmy patterson
3.99 7,030,084 362,437 800 distinct works
https://www.goodreads.com/book/show/3393124-max
309 pages March 16th 2009         by jimmy patterson
3.99 7,030,084 362,437 800 distinct works
https://www.goodreads.com/book/show/38501.Ham_on_Rye
288 pages July 29th 2014         by Ecco
3.97 666,546 30,741 423 distinct works
https://www.goodreads.com/book/show/38501.Ham_on_Rye
288 pages July 29th 2014         by Ecco
3.97 666,546 30,741 423 distinct works
https://www.goodreads.com/book/show/44778083-house

464 pages July 29th 2004         by Penguin Classics
4.03 252,264 12,524 519 distinct works
https://www.goodreads.com/book/show/28381.Dead_Souls
464 pages July 29th 2004         by Penguin Classics
4.03 252,264 12,524 519 distinct works
https://www.goodreads.com/book/show/31434883-eleanor-oliphant-is-completely-fine
336 pages May 9th 2017         by Pamela Dorman Books / Viking
4.26 933,680 76,929 5 distinct works
https://www.goodreads.com/book/show/31434883-eleanor-oliphant-is-completely-fine
336 pages May 9th 2017         by Pamela Dorman Books / Viking
4.26 933,680 76,929 5 distinct works
https://www.goodreads.com/book/show/27323.Hiroshima
152 pages March 4th 1989         by Vintage
3.99 83,255 4,616 40 distinct works
https://www.goodreads.com/book/show/27323.Hiroshima
152 pages March 4th 1989         by Vintage
3.99 83,255 4,616 40 distinct works
https://www.goodreads.com/book/show/6408862-stolen
304 pages May 4th 2009         by Chicken House Ltd
3.88 82,355 9,823 10 distinct work

4.18 1,775,850 86,225 1,440 distinct works
https://www.goodreads.com/book/show/431.The_New_York_Trilogy
308 pages March 28th 2006         by Penguin Classics
3.82 379,345 28,177 117 distinct works
https://www.goodreads.com/book/show/431.The_New_York_Trilogy
308 pages March 28th 2006         by Penguin Classics
3.82 379,345 28,177 117 distinct works
https://www.goodreads.com/book/show/13530588-china-blues
456 pages January 24th 2012         by Eio Books
3.92 27,959 1,453 14 distinct works
https://www.goodreads.com/book/show/13530588-china-blues
456 pages January 24th 2012         by Eio Books
3.92 27,959 1,453 14 distinct works
https://www.goodreads.com/book/show/7849341-tiger-s-voyage
560 pages November 1st 2011         by Hodder & Stoughton
4.17 186,683 17,143 21 distinct works
https://www.goodreads.com/book/show/7849341-tiger-s-voyage
560 pages November 1st 2011         by Hodder & Stoughton
4.17 186,683 17,143 21 distinct works
https://www.goodreads.com/book/show/261122.The_Dead_Gir

...

372 pages October 1st 2008         by Viking Books
3.92 451,459 40,463 31 distinct works
https://www.goodreads.com/book/show/20170404-station-eleven
333 pages September 9th 2014         by Knopf
3.95 522,429 60,375 8 distinct works
https://www.goodreads.com/book/show/20170404-station-eleven
333 pages September 9th 2014         by Knopf
3.95 522,429 60,375 8 distinct works
https://www.goodreads.com/book/show/38500.Women
291 pages July 29th 2014         by Ecco
3.97 666,475 30,748 476 distinct works
https://www.goodreads.com/book/show/38500.Women
291 pages July 29th 2014         by Ecco
3.97 666,475 30,748 476 distinct works
https://www.goodreads.com/book/show/89959.The_Constitution_of_the_United_States_of_America
30 pages January 1st 1970         by Applewood Books
4.49 51,031 1,246 66 distinct works
https://www.goodreads.com/book/show/89959.The_Constitution_of_the_United_States_of_America
30 pages January 1st 1970         by Applewood Books
4.49 51,031 1,246 66 distinct works
https://w

3.95 927,917 40,396 7 distinct works
https://www.goodreads.com/book/show/9418327-bossypants
283 pages April 5th 2011         by Reagan Arthur Books
3.95 927,917 40,396 7 distinct works
https://www.goodreads.com/book/show/52640.Weaveworld
721 pages August 20th 2001         by Harper Voyager
4.09 557,966 24,229 246 distinct works
https://www.goodreads.com/book/show/676737.Grendel
174 pages June 2nd 2010         by Random House Vintage Books
3.8 55,588 4,116 78 distinct works
https://www.goodreads.com/book/show/676737.Grendel
174 pages June 2nd 2010         by Random House Vintage Books
3.8 55,588 4,116 78 distinct works
https://www.goodreads.com/book/show/17349.The_Demon_Haunted_World
459 pages February 25th 1997         by Ballantine Books
4.23 455,141 15,572 50 distinct works
https://www.goodreads.com/book/show/9462812-die-for-me
341 pages May 10th 2011         by HarperTeen
4.01 127,304 11,482 19 distinct works
https://www.goodreads.com/book/show/9462812-die-for-me
341 pages May 10th 

https://www.goodreads.com/book/show/27833670-dark-matter
340 pages July 26th 2016         by Crown
4.05 805,986 83,542 48 distinct works
https://www.goodreads.com/book/show/27833670-dark-matter
340 pages July 26th 2016         by Crown
4.05 805,986 83,542 48 distinct works
https://www.goodreads.com/book/show/22544764-uprooted
435 pages May 19th 2015         by Del Rey
4.04 742,627 79,923 31 distinct works
https://www.goodreads.com/book/show/22544764-uprooted
435 pages May 19th 2015         by Del Rey
4.04 742,627 79,923 31 distinct works
https://www.goodreads.com/book/show/43945.Portnoy_s_Complaint
274 pages May 18th 1995         by Vintage
3.81 460,427 32,615 157 distinct works
https://www.goodreads.com/book/show/28249.The_Magicians_Guild
467 pages January 31st 2004         by Orbit
4.01 338,776 11,386 71 distinct works
https://www.goodreads.com/book/show/28249.The_Magicians_Guild
467 pages January 31st 2004         by Orbit
4.01 338,776 11,386 71 distinct works
https://www.goodreads.

383 pages May 2nd 2017         by Harper
4.0 1,427,538 82,130 64 distinct works
https://www.goodreads.com/book/show/22918050-the-heir
342 pages May 5th 2015         by HarperTeen
4.0 3,162,079 199,621 42 distinct works
https://www.goodreads.com/book/show/22918050-the-heir
342 pages May 5th 2015         by HarperTeen
4.0 3,162,079 199,621 42 distinct works
https://www.goodreads.com/book/show/1097.Fast_Food_Nation
399 pages July 5th 2005         by Harper Perennial
3.77 231,852 8,579 25 distinct works
https://www.goodreads.com/book/show/1097.Fast_Food_Nation
399 pages July 5th 2005         by Harper Perennial
3.77 231,852 8,579 25 distinct works
https://www.goodreads.com/book/show/284996.The_Complete_Stories
555 pages 1971         by Farrar, Straus & Giroux
4.14 176,694 12,920 151 distinct works
https://www.goodreads.com/book/show/284996.The_Complete_Stories
555 pages 1971         by Farrar, Straus & Giroux
4.14 176,694 12,920 151 distinct works
https://www.goodreads.com/book/show/634249

4.19 3,053,076 68,781 62 distinct works
https://www.goodreads.com/book/show/10037.No_Exit_and_Three_Other_Plays
275 pages 1989         by Vintage
3.99 347,585 14,959 551 distinct works
https://www.goodreads.com/book/show/26135825-fun-home
232 pages June 5th 2007         by Mariner Books
4.03 219,055 16,640 51 distinct works
https://www.goodreads.com/book/show/138872.Follow_the_River
406 pages November 12th 1986         by Ballantine Books
4.23 28,104 3,089 21 distinct works
https://www.goodreads.com/book/show/138872.Follow_the_River
406 pages November 12th 1986         by Ballantine Books
4.23 28,104 3,089 21 distinct works
https://www.goodreads.com/book/show/6574102-heist-society
287 pages February 9th 2010         by Little, Brown Books for Young Readers
4.05 788,554 50,345 45 distinct works
https://www.goodreads.com/book/show/6574102-heist-society
287 pages February 9th 2010         by Little, Brown Books for Young Readers
4.05 788,554 50,345 45 distinct works
https://www.goodreads.

In [478]:
soup = get_soup_proxy('https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo')

In [479]:
soup.find(itemprop='numberOfPages').text

'389 pages'

In [458]:
soup.find(class_='rating')

In [288]:
scrape_author_redo('Figures/get_year_page_count_author_attr.csv')

error


KeyboardInterrupt: 

In [380]:
path = 'Figures/good_reads_df.csv'
df = pd.read_csv(path, index_col=0)
df.columns

Index(['id_number', 'title', 'author', 'author_url', 'stars',
       'number of ratings', 'url', 'is_series', 'year_published', 'price',
       'number of pages', 'is_movie', 'movie title', 'movie year',
       'overdrive_url', 'media', 'Overdrive title', 'Wait time',
       'Library copies', 'People waiting in total', 'People waiting per copy',
       'Genres', 'overdrive_author'],
      dtype='object')

In [337]:
file = 'Figures/get_year_page_count_author_attr.csv'
author_attr = pd.read_csv(file, index_col=0)

In [305]:
df.drop_duplicates(subset=['id_number']).to_csv('Figures/test2.csv')

In [308]:
df.id_number = df.id_number.apply(lambda x: int(x))

In [ ]:
df.loc(df.author_url == 1, )

In [411]:
merged_df = df.merge(author_attr[['id_number', 'author_url','author_stars', 'author_ratings', 'author_num_reviews', 
                      'author_distinct_works']], on=['id_number'], how='left')#
merged_df.to_csv('Figures/merged_df.csv')
merged_df.loc[merged_df.author_url_x == '-1', 'author_url_x'] = merged_df.author_url_y
merged_df.drop('author_url_y', axis=1, inplace=True, errors='ignore')
merged_df.rename(columns = {'author_url_x':'author_url'}, inplace=True)
merged_df.to_csv('Figures/merged_df.csv')

merge author info

In [412]:
author_OD = pd.read_csv('Figures/get_author_OD.csv', index_col=0)

In [413]:
author_OD.id_number = author_OD.id_number.apply(lambda x: int(x))

In [414]:
merged_df = merged_df.merge(author_OD[['id_number', 'overdrive_author']], on=['id_number'], how='left')#.to_csv('Figures/test.csv')

In [424]:
merged_df.head(30)

,id_number,title,author,author_url,stars,number of ratings,url,is_series,year_published,price,...,Wait time,Library copies,People waiting in total,People waiting per copy,Genres,overdrive_author,author_stars,author_ratings,author_num_reviews,author_distinct_works
0,0.0,Night,Elie Wiesel,https://www.goodreads.com/author/show/1049.Eli...,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,12.99,...,0,5,-1,0,"['Biography & Autobiography', 'History', 'Nonf...",Elie Wiesel,4.32,"1,135,272","36,831",96 distinct works
2,0.0,Night,Elie Wiesel,https://www.goodreads.com/author/show/1049.Eli...,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,12.99,...,0,7,-6,0,"['Biography & Autobiography', 'Classic Literat...",Elie Wiesel,4.32,"1,135,272","36,831",96 distinct works
4,1.0,The Westing Game,Ellen Raskin,NaN,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,8.99,...,About 3 weeks,8,8,1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",Ellen Raskin,NaN,NaN,NaN,NaN
6,1.0,The Westing Game,Ellen Raskin,NaN,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,8.99,...,About 2 weeks,4,2,< 1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",Ellen Raskin,NaN,NaN,NaN,NaN
8,2.0,Hunted,P.C. Cast,NaN,3.91 avg rating,"178,631 ratings",/book/show/4134071-hunted,True,"March 14, 2017",1.99,...,0,2,-2,0,"['Comic and Graphic Books', 'Fiction']",Nick Spencer,NaN,NaN,NaN,NaN
10,2.0,Hunted,P.C. Cast,NaN,3.91 avg rating,"178,631 ratings",/book/show/4134071-hunted,True,"March 14, 2017",1.99,...,info unavailable,info unavailable,info unavailable,info unavailable,"['Fantasy', 'Juvenile Fiction']",Nick Spencer,NaN,NaN,NaN,NaN
12,3.0,We Need to Talk About Kevin,Lionel Shriver,NaN,4.04 avg rating,"188,422 ratings",/book/show/80660.We_Need_to_Talk_About_Kevin,False,"April 14, 2003",14.99,...,0,12,-6,0,"['Fiction', 'Literature']",Lionel Shriver,NaN,NaN,NaN,NaN
14,3.0,We Need to Talk About Kevin,Lionel Shriver,NaN,4.04 avg rating,"188,422 ratings",/book/show/80660.We_Need_to_Talk_About_Kevin,False,"April 14, 2003",14.99,...,About 8 weeks,3,11,4,"['Fiction', 'Literature']",Lionel Shriver,NaN,NaN,NaN,NaN
16,4.0,The Bean Trees,Barbara Kingsolver,NaN,3.98 avg rating,"140,989 ratings",/book/show/30868.The_Bean_Trees,True,1988,7.99,...,0,3,-2,0,"['Fiction', 'Literature']",Barbara Kingsolver,NaN,NaN,NaN,NaN
18,4.0,The Bean Trees,Barbara Kingsolver,NaN,3.98 avg rating,"140,989 ratings",/book/show/30868.The_Bean_Trees,True,1988,7.99,...,0,2,-1,0,"['Fiction', 'Literature']",Barbara Kingsolver,NaN,NaN,NaN,NaN


In [416]:
merged_df.loc[merged_df['overdrive_author_x'].isna(), 'overdrive_author_x'] = merged_df.overdrive_author_y
merged_df.drop('overdrive_author_y', axis=1, inplace=True, errors='ignore')
merged_df.rename(columns = {'overdrive_author_x':'overdrive_author'}, inplace=True)
merged_df.to_csv('Figures/merged_df.csv')

In [420]:
merged_df.loc[merged_df.price.notna(),:] = merged_df.loc[merged_df.price.notna(),:].drop_duplicates(subset = ['overdrive_url'])

In [422]:
merged_df.dropna(inplace=True,how='all')

In [423]:
merged_df.to_csv('Figures/merged_df.csv')

In [431]:
merged_df[merged_df.author_url.isna()].index

Int64Index([   4,    6,    8,   10,   12,   14,   16,   18,   20,   22,
            ...
            4132, 4134, 4136, 4138, 4140, 4142, 4144, 4146, 4148, 4150],
           dtype='int64', length=1496)

In [432]:
merged_df[(merged_df.author_url.isna()) | (merged_df.year_published == "'NoneType' object has no attribute 'parent'") | (merged_df['number of pages'] == "'NoneType' object has no attribute 'parent'")]

,id_number,title,author,author_url,stars,number of ratings,url,is_series,year_published,price,...,Wait time,Library copies,People waiting in total,People waiting per copy,Genres,overdrive_author,author_stars,author_ratings,author_num_reviews,author_distinct_works
0,0.0,Night,Elie Wiesel,https://www.goodreads.com/author/show/1049.Eli...,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,12.99,...,0,5,-1,0,"['Biography & Autobiography', 'History', 'Nonf...",Elie Wiesel,4.32,"1,135,272","36,831",96 distinct works
2,0.0,Night,Elie Wiesel,https://www.goodreads.com/author/show/1049.Eli...,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,12.99,...,0,7,-6,0,"['Biography & Autobiography', 'Classic Literat...",Elie Wiesel,4.32,"1,135,272","36,831",96 distinct works
4,1.0,The Westing Game,Ellen Raskin,NaN,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,8.99,...,About 3 weeks,8,8,1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",Ellen Raskin,NaN,NaN,NaN,NaN
6,1.0,The Westing Game,Ellen Raskin,NaN,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,8.99,...,About 2 weeks,4,2,< 1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",Ellen Raskin,NaN,NaN,NaN,NaN
8,2.0,Hunted,P.C. Cast,NaN,3.91 avg rating,"178,631 ratings",/book/show/4134071-hunted,True,"March 14, 2017",1.99,...,0,2,-2,0,"['Comic and Graphic Books', 'Fiction']",Nick Spencer,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5244,1608.0,NOS4A2,Joe Hill,https://www.goodreads.com/author/show/88506.Jo...,4.08 avg rating,"113,369 ratings",/book/show/15729539-nos4a2,False,30 April 2013,12.99,...,About 8 weeks,2,7,4,"['Fiction', 'Horror', 'Literature', 'Thriller']",Joe Hill,NaN,NaN,NaN,NaN
5249,1611.0,No Exit and Three Other Plays,Jean-Paul Sartre,https://www.goodreads.com/author/show/1466.Jea...,4.06 avg rating,"29,249 ratings",/book/show/10037.No_Exit_and_Three_Other_Plays,False,1944,11.99,...,0,3,-2,0,"['Fiction', 'Literary Anthologies']",Jean-Paul Sartre,NaN,NaN,NaN,NaN
5265,1620.0,El túnel,Ernesto Sabato,https://www.goodreads.com/author/show/5775606....,4.04 avg rating,"54,671 ratings",/book/show/53447.El_t_nel,False,1948,14.99,...,0,2,-1,0,"['Fiction', 'Literature']",Ernesto Sabato,NaN,NaN,NaN,NaN
5266,1620.0,El túnel,Ernesto Sabato,https://www.goodreads.com/author/show/5775606....,4.04 avg rating,"54,671 ratings",/book/show/53447.El_t_nel,False,1948,14.99,...,0,1,-1,0,"['Juvenile Fiction', 'Juvenile Literature', 'S...",Ana Campoy,NaN,NaN,NaN,NaN


In [239]:
soup.find(text = re.compile('first published'))

'\n              (first published 1956)\n            '

In [335]:
df[(df['Library copies'].notna()) & (df['Library copies'] != 'info unavailable') & (df['Library copies'] != 'title not found')]#.to_csv('Figures/test_df.csv')

,id_number,title,author,author_url,stars,number of ratings,url,is_series,year_published,price,...,movie year,overdrive_url,media,Overdrive title,Wait time,Library copies,People waiting in total,People waiting per copy,Genres,overdrive_author
0,0,Night,Elie Wiesel,-1,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,12.99,...,2021,/media/3946821,Audiobook,Night,0,5,-1,0,"['Biography & Autobiography', 'History', 'Nonf...",NaN
1,0,Night,Elie Wiesel,-1,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,12.99,...,2021,/media/90613,Audiobook,Night,0,7,-6,0,"['Biography & Autobiography', 'Classic Literat...",NaN
2,1,The Westing Game,Ellen Raskin,-1,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,8.99,...,1997,/media/283610,ebook,The Westing Game,About 3 weeks,8,8,1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",NaN
3,1,The Westing Game,Ellen Raskin,-1,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,8.99,...,1997,/media/3326221,Audiobook,The Westing Game,About 2 weeks,4,2,< 1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",NaN
4,2,Hunted,P.C. Cast,-1,3.91 avg rating,"178,631 ratings",/book/show/4134071-hunted,True,"March 14, 2017",1.99,...,1997,/media/5408082,ebook,"The Amazing Spider-Man by Nick Spencer, Volume 4",0,2,-2,0,"['Comic and Graphic Books', 'Fiction']",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,1613,Follow the River,James Alexander Thom,https://www.goodreads.com/author/show/29544.Ja...,4.22 avg rating,"18,935 ratings",/book/show/138872.Follow_the_River,False,1981,7.99,...,NaN,/media/106979,ebook,"Sail Away, Little Boat",0,1,-1,0,"['Juvenile Fiction', 'Juvenile Literature']",Janet Buell
3197,1614,Heist Society,Ally Carter,https://www.goodreads.com/author/show/56224.Al...,3.90 avg rating,"93,578 ratings",/book/show/6574102-heist-society,True,"February 9, 2010",8.99,...,NaN,/media/520974,ebook,Heist Society,0,4,-3,0,"['Young Adult Fiction', 'Young Adult Literature']",Ally Carter
3198,1614,Heist Society,Ally Carter,https://www.goodreads.com/author/show/56224.Al...,3.90 avg rating,"93,578 ratings",/book/show/6574102-heist-society,True,"February 9, 2010",8.99,...,NaN,/media/3980794,Audiobook,The Gilded Wolves--A Novel,About 2 weeks,4,3,< 1,"['Fantasy', 'Historical Fiction', 'Young Adult...",Roshani Chokshi
3199,1615,1776,David McCullough,https://www.goodreads.com/author/show/6281688....,4.06 avg rating,"212,232 ratings",/book/show/1067.1776,False,"May 24, 2005",13.99,...,1972,/media/79158,Audiobook,1776,0,6,-2,0,"['Biography & Autobiography', 'History', 'Mili...",David McCullough


In [524]:
df = pd.read_csv('Figures/merged_df.csv', index_col=0)
df[(df.author_url.isna()) | ((df.author_stars.isna()) & (df.year_published.notna()))]

,id_number,title,author,author_url,stars,number of ratings,url,is_series,year_published,price,...,Wait time,Library copies,People waiting in total,People waiting per copy,Genres,overdrive_author,author_stars,author_ratings,author_num_reviews,author_distinct_works
4154,1001.0,This Side of Paradise,F. Scott Fitzgerald,https://www.goodreads.com/author/show/3190.F_S...,3.65 avg rating,"68,250 ratings",/book/show/46165.This_Side_of_Paradise,False,"March 26, 1920",0.99,...,0,4,-3,0,"['Classic Literature', 'Fiction']",-1,NaN,NaN,NaN,NaN
4155,1001.0,This Side of Paradise,F. Scott Fitzgerald,https://www.goodreads.com/author/show/3190.F_S...,3.65 avg rating,"68,250 ratings",/book/show/46165.This_Side_of_Paradise,False,"March 26, 1920",0.99,...,0,Always available,Unknown,0,"['Essays', 'Fiction', 'Language Arts']",F. Scott Fitzgerald,NaN,NaN,NaN,NaN
4158,1003.0,Defending Jacob,William Landay,https://www.goodreads.com/author/show/218843.W...,3.98 avg rating,"286,315 ratings",/book/show/11367726-defending-jacob,False,"January 31, 2012",11.99,...,0,64,-63,0,"['Fiction', 'Literature', 'Suspense', 'Thriller']",-1,NaN,NaN,NaN,NaN
4159,1003.0,Defending Jacob,William Landay,https://www.goodreads.com/author/show/218843.W...,3.98 avg rating,"286,315 ratings",/book/show/11367726-defending-jacob,False,"January 31, 2012",11.99,...,0,20,-18,0,"['Fiction', 'Mystery']",William Landay,NaN,NaN,NaN,NaN
4160,1004.0,Dead in the Family,Charlaine Harris,https://www.goodreads.com/author/show/17061.Ch...,3.88 avg rating,"160,468 ratings",/book/show/7091488-dead-in-the-family,True,2010,8.99,...,0,16,-15,0,"['Fantasy', 'Fiction', 'Mystery', 'Romance']",-1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5263,1619.0,The Fifth Season,N.K. Jemisin,https://www.goodreads.com/author/show/2917917....,4.28 avg rating,"203,855 ratings",/book/show/19161852-the-fifth-season,True,"August 4, 2015",9.99,...,Available soon,56,20,< 1,"['Fantasy', 'Fiction', 'Literature', 'Thriller']",N. K. Jemisin,NaN,NaN,NaN,NaN
5264,1619.0,The Fifth Season,N.K. Jemisin,https://www.goodreads.com/author/show/2917917....,4.28 avg rating,"203,855 ratings",/book/show/19161852-the-fifth-season,True,"August 4, 2015",9.99,...,info unavailable,info unavailable,info unavailable,info unavailable,"['Fantasy', 'Fiction', 'Literature']",N. K. Jemisin,NaN,NaN,NaN,NaN
5265,1620.0,El túnel,Ernesto Sabato,https://www.goodreads.com/author/show/5775606....,4.04 avg rating,"54,671 ratings",/book/show/53447.El_t_nel,False,1948,14.99,...,0,2,-1,0,"['Fiction', 'Literature']",Ernesto Sabato,NaN,NaN,NaN,NaN
5266,1620.0,El túnel,Ernesto Sabato,https://www.goodreads.com/author/show/5775606....,4.04 avg rating,"54,671 ratings",/book/show/53447.El_t_nel,False,1948,14.99,...,0,1,-1,0,"['Juvenile Fiction', 'Juvenile Literature', 'S...",Ana Campoy,NaN,NaN,NaN,NaN
